# Data Cleaning

This notebook clean the data to prepare it for the analysis. The cleaned data is then save to df_clean.csv

We will do the follows. 

1. Remove variables(columns) that has more than 70% of NaN value

2.  Handle categorical values with too many categories: pt_state, ecodub92 and diag_adm:  We will find a reasonable way to group these categories into fewer categories. 

3. Save the data to df_clean.csv
 

In [2]:
import pandas as pd
import numpy as np

from collections import defaultdict

In [2]:
df = pd.read_csv('../df.csv')

D:\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (5,6,8,9,16,20) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


### 1. Remove variables 

We remove variables that has more than 70% of missing values. 

In [3]:
DROP_LIMIT = 0.7
for var in df.columns:
    if df[var].isna().sum()/df[var].count() > DROP_LIMIT:
        df.drop(var, axis=1, inplace=True)

In [4]:
orig_col = ['yod','age','sex','b_wt','ethnic','pt_state','race','raceethn','campus','er_mode','admtype', 'payer',
      'yoa','pay_ub92','provider','asource','ecodepoa','moa','service','ecodub92','diag_adm','los']

for c in orig_col:
    if c not in df.columns:
        print(c)

race
er_mode
ecodepoa


#### Removed columns:
* race
* er_mode
* ecodepoa

### 2. Handle pt_state

In [5]:
df['pt_state'].unique()

array([nan, "b'00'", "b'RI'", "b'CA'", "b'01'", "b'08'", "b'TX'", "b'CT'",
       "b'NH'", "b'02'", "b'NC'", "b'DE'", "b'YY'", "b'MA'", "b'NY'",
       "b'05'", "b'25'", "b'26'", "b'20'", "b'31'", "b'FL'", "b'11'",
       "b'06'", "b'18'", "b'PA'", "b'03'", "b'16'", "b'IL'", "b'10'",
       "b'13'", "b'21'", "b'07'", "b'09'", "b'LA'", "b'17'", "b'12'",
       "b'NE'", "b'ME'", "b'04'", "b'22'", "b'NJ'", "b'15'", "b'19'",
       "b'34'", "b'14'", "b'MI'", "b'MD'", "b'OR'", "b'NV'", "b'27'",
       "b'OH'", "b'GA'", "b'AK'", "b'53'", "b'E8'", "b'MO'", "b'IN'",
       "b'TN'", "b'VT'", "b'CO'", "b'AZ'", "b'VA'", "b'WI'", "b'KS'",
       "b'-2'", "b'SC'", "b'DC'", "b'30'", "b'V4'", "b'MN'", "b'UK'",
       "b'OK'", "b'V1'", "b'24'", "b'44'", "b'36'", "b'40'", "b'33'",
       "b'61'", "b'80'", "b'45'", "b'73'", "b'AL'", "b'HI'", "b'0-'",
       "b'XX'", "b'KY'", "b'E9'", "b'78'", "b'42'", "b'69'", "b'V0'",
       "b'74'", "b'23'", "b'-3'", "b'WV'", "b'VI'", "b'AR'", "b'UT'",
       "b'28'",

In [5]:
map_dict = defaultdict(lambda: 'Other')
map_dict["b'RI'"] = 'RI'
map_dict["b'CT'"] = 'CT'
map_dict["b'MA'"] = 'MA'

In [6]:
df['pt_state'] = df['pt_state'].map(map_dict, na_action='ignore').fillna('Unknown')

In [7]:
df['pt_state'].unique()

array(['Unknown', 'Other', 'RI', 'CT', 'MA'], dtype=object)

### 3. Handle ecodub92


In [8]:
df = df.drop(['ecodub92'], axis=1)

### 4. Handle diag_adm

In [9]:
# fill NA values with 'Unknown'
df['diag_adm'] = df['diag_adm'].fillna('Unknown')

# Keep the first three letters of the diagnosis (Categories of dianosis)
df['diag_adm'] = [i if i=='Unknown' else i[2:5] for i in df['diag_adm']]

# Group diagnosis codes with few values into Others
u = df['diag_adm'].value_counts()
v = u.index[u.values<5]
df['diag_adm'] = ['Others' if i in v else i for i in df['diag_adm']]

In [10]:
def f(x):
    r = pd.Series()
    r['Unique Values'] =  x.unique().__len__()
    missing_values = x.isna().sum()
    r['Missing'] = missing_values
    r['Missing Percentage'] =  missing_values/len(x)
    r['Type'] = x.dtype
    return(r)

In [11]:
u = df.apply(f, axis=0)
u.transpose()

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  


,Unique Values,Missing,Missing Percentage,Type
yod,16,0,0,float64
age,120,25,1.13046e-05,float64
sex,4,3,1.35655e-06,object
b_wt,4847,590017,0.266796,object
ethnic,6,285208,0.128966,float64
pt_state,5,0,0,object
raceethn,10,146005,0.0660211,object
campus,34,903208,0.408416,object
admtype,7,75,3.39138e-05,object
payer,18,0,0,object


### 5. Save the data

In [13]:
df.to_csv('../df_clean.csv', index=False)

## Further Cleaning and resaving

In [3]:
df = pd.read_csv('../df_clean.csv')

D:\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
column_values = []
for c in df.columns:
    column_values.append((c, df[c].dtype, df[c].unique()))

In [6]:
column_values

[('yod',
  dtype('float64'),
  array([ 3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12., 13., 14., 15.,
         16., 17., 18.])),
 ('age',
  dtype('float64'),
  array([ 39.,  59.,  57.,  54.,  58.,  48.,  50.,  55.,  46.,  56.,  76.,
          47.,  61.,  45.,  51.,  42.,  63.,  64.,  62.,  43.,  84.,  67.,
          37.,  38.,  41.,  70.,  35.,  49.,  40.,  33.,  36.,  32.,  52.,
          44.,  60.,  53.,  71.,  28.,  78.,  26.,  30.,  34.,  23.,  25.,
          22.,  21.,  27.,  29.,  24.,  81.,  20.,  19.,  14.,  31.,  75.,
          82.,  13.,  15.,  18.,  10.,  17.,  12.,  11.,  68.,  85.,  80.,
          98.,   9.,   8.,  79.,  86.,  65.,  16.,  88.,  83.,  77.,  73.,
           7.,  95.,   5.,  66.,  89.,   4.,  72.,  69.,   6.,  91.,  87.,
          94.,  74.,  90.,   3.,  93.,  nan,  92.,  96.,   0.,   1., 100.,
           2.,  97., 104.,  99., 101., 102., 114., 105., 106., 108., 103.,
         115., 109., 107., 118., 119., 122., 110., 111., 123., 124.])),
 ('sex', dtype('O')

### Mapping yod (year of discharge), yoa (year of admission), and moa (month of admission) to int
* Currently it is the last two digits of the year, with leading zeroes truncated
* Possible format to full year?

In [5]:
df['yod'] = pd.to_numeric(df['yod'], downcast='integer')
df['yoa'] = pd.to_numeric(df['yoa'], downcast='integer')
df['moa'] = pd.to_numeric(df['moa'].fillna(-1), downcast='integer')

In [6]:
df['yod'].unique()

array([ 3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18],
      dtype=int8)

In [7]:
df['yoa'].unique()

array([ 3,  2,  1,  4,  0, 96,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15,
       16, 17, 18], dtype=int8)

In [8]:
df['moa'].unique()

array([ 2,  1,  3, 12,  7,  9, 11,  4,  5,  6,  8, 10, -1,  0], dtype=int8)

### Mapping age to int
* NaN mapped to -1

In [9]:
df['age'] = pd.to_numeric(df['age'].fillna(-1), downcast='integer')

In [10]:
np.sort(df['age'].unique())

array([ -1,   0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,
        12,  13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,
        25,  26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,
        38,  39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,
        51,  52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,
        64,  65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,
        77,  78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,
        90,  91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102,
       103, 104, 105, 106, 107, 108, 109, 110, 111, 114, 115, 118, 119,
       122, 123, 124], dtype=int8)

### Handling sex
* Changing b'1' and b'2' to 1 and 2 (Male and Female respectively)
* Documentation does not explain b'9', so changing that and nan to -1

In [11]:
map_dict = {"b'1'": 'Male', "b'2'": 'Female', "b'9'": 'Unknown'}

In [12]:
df['sex'] = df['sex'].map(map_dict, na_action='ignore').fillna('Unknown').astype('category')

In [13]:
df['sex'].unique()

[Female, Male, Unknown]
Categories (3, object): [Female, Male, Unknown]

### Making b_wt (weight in grams when born) numeric
* Making nan values -1
* Changing values not fully numeric (e.g. b'0V58') to -1 as well
    * Some values possibly only missing certain digits (e.g. b'58XX') could have digits filled if desired

In [14]:
def map_func(x):
    try:
        return int(x[2:-1])
    except:
        return -1

In [15]:
df['b_wt'] = df['b_wt'].map(map_func, na_action='ignore').fillna(-1)
df['b_wt'] = pd.to_numeric(df['b_wt'], downcast='integer')

In [16]:
df['b_wt'].unique()

array([  -1, 5382, 8121, ...,  132,  358, 8349], dtype=int16)

### Creating Stay column
* Short is los >=0 and <= 14
* Long is los > 14
* Neither is los < 0 and nan values

In [17]:
df['Stay'] = pd.cut(df['los'].fillna(-1),
                    bins=[df['los'].min()-1, -1, 14, df['los'].max()], 
                    labels=['Neither', 'Short', 'Long']
                   ).astype('category')

In [18]:
df['Stay'].unique()

[Short, Long, Neither]
Categories (3, object): [Neither < Short < Long]

### Fixing negative los values
* Negative and nan values defaulting to -1

In [19]:
def remove_neg_values(num):
    if num < 0:
        return -1
    else:
        return num

In [20]:
df['los'] = pd.to_numeric(df['los'].fillna(-1).map(remove_neg_values), downcast='integer')

### Handling raceethn

In [21]:
map_dict = {"b'1'": 'White, not Hispanic', "b'2'": 'Black, not Hispanic', "b'3'": 'Asian, not Hispanic', 
            "b'4'": 'American Indian, not Hispanic', "b'5'": 'Native Hawaiian, not Hispanic', "b'6'": 'Other, not Hispanic', 
            "b'7'": 'Hispanic, all races', "b'9'": 'Unknown', "b'0'": 'Unknown'}

In [22]:
df['raceethn'] = df['raceethn'].map(map_dict, na_action='ignore').fillna('Unknown').astype('category')

In [23]:
df['raceethn'].unique()

[White, not Hispanic, Black, not Hispanic, Other, not Hispanic, Asian, not Hispanic, American Indian, not Hispanic, Native Hawaiian, not Hispanic, Unknown, Hispanic, all races]
Categories (8, object): [White, not Hispanic, Black, not Hispanic, Other, not Hispanic, Asian, not Hispanic, American Indian, not Hispanic, Native Hawaiian, not Hispanic, Unknown, Hispanic, all races]

Labeling could probably use some work

### Handling ethnic

In [24]:
map_dict = {0: 'Unknown', 1:'Yes hispanic', 2:'Not hispanic', 
            3:'Not reported', 9:'Not reported'}

In [26]:
df['ethnic'] = df['ethnic'].map(map_dict, na_action='ignore').fillna('Unknown').astype('category')

In [27]:
df['ethnic'].unique()

[Unknown, Not hispanic, Not reported, Yes hispanic]
Categories (4, object): [Unknown, Not hispanic, Not reported, Yes hispanic]

Not reported and unknown could probably be combined

### Handling admtype

In [28]:
map_dict = {"b'1'": 'Emergency', "b'2'": 'Urgent', "b'3'": 'Electric', \
            "b'4'": 'Newborn', "b'5'": 'Trauma', "b'9'": 'Information not available'}

In [29]:
df['admtype'] = df['admtype'].map(map_dict, na_action='ignore').fillna('Information not available').astype('category')

In [30]:
df['admtype'].unique()

[Electric, Emergency, Trauma, Urgent, Newborn, Information not available]
Categories (6, object): [Electric, Emergency, Trauma, Urgent, Newborn, Information not available]

What exactly is electric?

### Handling provider

In [31]:
map_dict = {"b'7216'": 'Butler', "b'7215'": 'Bradley', "b'7212'": 'Rehab of RI', "b'7201'": 'Newport', 
            "b'7209'": 'South County', "b'7203'": 'Memorial', "b'7205'": 'Rhode Island Hospital', 
            "b'7204'": 'Miriam', "b'7210'": 'Kent County', "b'7206'": 'Roger Williams', 
            "b'7213'": 'Landmark Medical Center', "b'7202'": 'St. Joseph Health Services of RI',
            "b'7214'": 'Women and Infants', "b'7211'": 'Westerly'}

In [32]:
df['provider'] = df['provider'].map(map_dict, na_action='ignore').fillna('Unknown').astype('category')

In [33]:
df['provider'].unique()

[Butler, Bradley, Rehab of RI, Newport, South County, ..., Roger Williams, Landmark Medical Center, St. Joseph Health Services of RI, Women and Infants, Westerly]
Length: 14
Categories (14, object): [Butler, Bradley, Rehab of RI, Newport, ..., Landmark Medical Center, St. Joseph Health Services of RI, Women and Infants, Westerly]

### Handling service

In [34]:
map_dict = {"b'00'": 'Unknown', "b'02'": 'Pediactrics', "b'10'": 'Medicine', "b'22'": 'Cardiology', 
            "b'38'": 'Psychiatry', "b'40'": 'Surgery', "b'48'": 'Ophthalmology', "b'50'": 'ENT', 
            "b'54'": 'Oral Surgery', "b'58'": 'Orthopedics', "b'62'": 'Urology', "b'70'": 'Gynecology', 
            "b'75'": 'Abortion', "b'76'": 'OB – Not Delivered', "b'77'": 'OB – Delivered', 
            "b'80'": 'Newborn', "b'98'": 'Rehabilitation'}

In [35]:
df['service'] = df['service'].map(map_dict, na_action='ignore').fillna('Unknown').astype('category')

In [36]:
df['service'].unique()

[Psychiatry, Rehabilitation, Medicine, Surgery, Orthopedics, ..., Ophthalmology, Oral Surgery, ENT, Abortion, Unknown]
Length: 17
Categories (17, object): [Psychiatry, Rehabilitation, Medicine, Surgery, ..., Oral Surgery, ENT, Abortion, Unknown]

### Saving

In [37]:
df.to_csv('..\df_clean2.csv', index=False)